<a href="https://colab.research.google.com/github/alexander-toschev/ml-cs-intro/blob/main/home-work/HW_END_TO_END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Deep Learning with Keras — Homework

This notebook contains **auto-graded tasks** about end-to-end deep learning with Keras.

Fill in the cells marked with `# TODO` and run the test cells below each task.

- Language: English (code + comments)
- Topic focus: **end-to-end deep learning** with CNNs in Keras
- Total: **100 points**

After each test cell you will see:
- Points for this task
- Cumulative **TOTAL POINTS**


In [1]:
# @title 1) Student Info & Config
# All code comments are in English.


# === ОБЯЗАТЕЛЬНО ЗАПОЛНИТЬ ===
full_name = "Doe John"     # например: "Тощев Александр"
student_group = "11-111"      # например: "208"
assignment_id = "HW_END-TO_END"
assert full_name != "Фамилия Имя", "Заполните full_name"
assert student_group != "Группа", "Заполните student_group"
print("✔ Student Info OK")

# Typical human accuracy (benchmark) for MNIST may be ~97-99%.

print("Student:", full_name)

✔ Student Info OK
Student: Doe John


In [2]:

import numpy as np

def _close(a, b, tol=1e-8):
    return np.allclose(a, b, atol=tol)

def _arr_equal(a, b):
    a = np.array(a)
    b = np.array(b)
    return a.shape == b.shape and np.array_equal(a, b)

In [3]:
from datetime import datetime, timezone, timedelta

# Установите окна приёма (пример):

start_at_iso = "2025-11-24T09:00-04:00"  #@param {type:"string"}
due_at_iso   = "2025-12-01T23:59-04:00"  #@param {type:"string"}
start_dt = datetime.fromisoformat(start_at_iso)
due_dt   = datetime.fromisoformat(due_at_iso)
# Для протокола: время сдачи берём текущее (можно заменить на mtime файла)
import os
from datetime import datetime, timezone

# 📅 Add submission date based on file modification time
try:
    nb_path = __file__ if "__file__" in globals() else "EM01_Numpy.ipynb"
    mtime = os.path.getmtime(nb_path)
    submission_dt = datetime.fromtimestamp(mtime, tz=timezone.utc)
except Exception:
    submission_dt = datetime.utcnow().replace(tzinfo=timezone.utc)

def penalty_fraction(start_dt, due_dt, submission_dt):
    """Возвращает долю штрафа [0..1].
    0 — без штрафа (<= due_dt). Линейно растёт от due_dt к due_dt + (due_dt - start_dt).
    Не выходит за 1.0.
    """
    if submission_dt <= due_dt:
        return 0.0
    total = (due_dt - start_dt).total_seconds()
    late  = (submission_dt - due_dt).total_seconds()
    if total <= 0:
        return 1.0 if late > 0 else 0.0
    return min(1.0, max(0.0, late / total))

print(f"Окно приёма: {start_dt.isoformat()} — {due_dt.isoformat()} (UTC)")
print(f"Время сдачи: {submission_dt.isoformat()} (UTC)")

Окно приёма: 2025-11-24T09:00:00-04:00 — 2025-12-01T23:59:00-04:00 (UTC)
Время сдачи: 2025-12-01T15:38:30.979113+00:00 (UTC)


/tmp/ipython-input-632059888.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  submission_dt = datetime.utcnow().replace(tzinfo=timezone.utc)


In [4]:
# Global score storage (do not modify)
SCORES = {}

def _set_score(task_name, points, max_points):
    SCORES[task_name] = min(points, max_points)
    total = sum(SCORES.values())
    print(f"Task {task_name}: {SCORES[task_name]} / {max_points} points")
    print(f"TOTAL POINTS: {total} / 100\n")


## 0. Imports and data loading

In this homework we will use **Fashion-MNIST** as an example dataset.
It is slightly harder than MNIST but still small enough to train quickly in Colab.

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

# Load Fashion-MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Add channel dimension: (N, 28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

num_classes = 10
input_shape = x_train.shape[1:]

print('Train shape:', x_train.shape)
print('Test shape:', x_test.shape)
print('Input shape:', input_shape)


2.19.0
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (60000, 28, 28, 1)
Test shape: (10000, 28, 28, 1)
Input shape: (28, 28, 1)


## Task 1 (40 points): Build an end-to-end CNN model

Implement the function `build_e2e_cnn(input_shape, num_classes)`.

Requirements:
- Use `tf.keras` / `keras` only.
- The model must be **end-to-end**:
  - Input: raw normalized images of shape `input_shape` (e.g., `(28, 28, 1)`).
  - Output: logits or probabilities over `num_classes` classes.
- Architecture constraints:
  - Use at least **2 convolutional layers** (`Conv2D`).
  - Use at least **1 pooling layer** (`MaxPooling2D`).
  - Use at least **1 Dense hidden layer** before the final output layer.
- The final layer must have `num_classes` units.
- The model must be **compiled** with:
  - optimizer: `'adam'`
  - loss: `'sparse_categorical_crossentropy'`
  - metric: `'accuracy'`

Feel free to change the number of filters/units, as long as the constraints above are satisfied.

In [6]:
# TODO: implement build_e2e_cnn
def build_e2e_cnn(input_shape, num_classes):
    """Build and compile an end-to-end CNN model.

    Args:
        input_shape: tuple, e.g. (28, 28, 1)
        num_classes: int, e.g. 10

    Returns:
        A compiled tf.keras.Model instance.
    """
    # YOUR CODE HERE
    raise NotImplementedError


In [7]:
# TEST CELL: Task 1 (40 points)
max_points = 40
points = 0

try:
    model_t1 = build_e2e_cnn(input_shape, num_classes)
except Exception as e:
    print('Error when calling build_e2e_cnn:', e)
    _set_score('1', points, max_points)
else:
    # Check type
    if isinstance(model_t1, keras.Model):
        points += 5
    else:
        print('Model is not an instance of keras.Model')

    # Check compilation
    if model_t1.optimizer is not None:
        points += 5

    # Check loss and metrics
    loss = getattr(model_t1, 'loss', None)
    metrics = getattr(model_t1, 'metrics', [])
    if isinstance(loss, str) and 'sparse_categorical_crossentropy' in loss:
        points += 5
    elif hasattr(loss, '__name__') and 'sparse_categorical_crossentropy' in loss.__name__:
        points += 5
    else:
        print('Unexpected loss function (should be sparse_categorical_crossentropy).')

    metric_names = [m.name for m in metrics if hasattr(m, 'name')]
    if 'accuracy' in metric_names or 'acc' in metric_names:
        points += 5
    else:
        print('No accuracy metric found.')

    # Check final layer size
    try:
        output_units = model_t1.output_shape[-1]
        if output_units == num_classes:
            points += 10
        else:
            print(f'Output units {output_units} != num_classes {num_classes}')
    except Exception as e:
        print('Error checking output shape:', e)

    # Check presence of Conv2D, MaxPooling2D, Dense
    layer_types = [type(l) for l in model_t1.layers]
    conv_count = sum(issubclass(t, layers.Conv2D) for t in layer_types)
    pool_count = sum(issubclass(t, layers.MaxPooling2D) for t in layer_types)
    dense_count = sum(issubclass(t, layers.Dense) for t in layer_types)

    if conv_count >= 2:
        points += 5
    else:
        print('Model must have at least 2 Conv2D layers.')

    if pool_count >= 1:
        points += 3
    else:
        print('Model must have at least 1 MaxPooling2D layer.')

    if dense_count >= 2:  # hidden + output
        points += 2
    else:
        print('Model must have at least 1 hidden Dense layer + output Dense layer.')

    _set_score('1', points, max_points)


Error when calling build_e2e_cnn: 
Task 1: 0 / 40 points
TOTAL POINTS: 0 / 100



## Task 2 (30 points): Train the end-to-end CNN

Now implement `train_e2e_cnn` that trains your model on Fashion-MNIST.

Requirements:
- Split the training set into train/validation using `validation_split` or manual split.
- Train for **at least 3 epochs**.
- Return the **History** object from `model.fit`.

Do **not** hard-code global variables inside the function body (use arguments).

In [8]:
# TODO: implement train_e2e_cnn
def train_e2e_cnn(model, x_train, y_train, epochs=3, batch_size=64, validation_split=0.1):
    """Train the end-to-end CNN model on Fashion-MNIST.

    Args:
        model: compiled keras.Model
        x_train, y_train: training data and labels
        epochs: number of epochs (>= 3)
        batch_size: batch size
        validation_split: fraction of training data for validation

    Returns:
        history: the History object returned by model.fit
    """
    # YOUR CODE HERE
    raise NotImplementedError


In [9]:
# TEST CELL: Task 2 (30 points)
max_points = 30
points = 0

try:
    model_t2 = build_e2e_cnn(input_shape, num_classes)
    history_t2 = train_e2e_cnn(model_t2, x_train, y_train, epochs=3, batch_size=128, validation_split=0.1)
except Exception as e:
    print('Error in train_e2e_cnn or during training:', e)
    _set_score('2', points, max_points)
else:
    # Check type
    if hasattr(history_t2, 'history'):
        points += 10
    else:
        print('Returned object does not look like a Keras History.')

    # Check presence of accuracy and loss in logs
    h = history_t2.history
    keys = h.keys()
    if 'loss' in keys and any(k.startswith('val_') and 'loss' in k for k in keys):
        points += 5
    else:
        print('No loss/val_loss in history.')

    if 'accuracy' in keys or 'acc' in keys:
        points += 5
    else:
        print('No accuracy in history.')

    # Check number of epochs
    epochs_run = len(h.get('loss', []))
    if epochs_run >= 3:
        points += 10
    else:
        print(f'Not enough epochs trained: {epochs_run} < 3')

    _set_score('2', points, max_points)


Error in train_e2e_cnn or during training: 
Task 2: 0 / 30 points
TOTAL POINTS: 0 / 100



## Task 3 (30 points): Decide if a pipeline is end-to-end

Implement the function `is_end_to_end` that decides whether a given pipeline is **end-to-end deep learning**.

The pipeline is described by three boolean flags:
- `has_raw_input_to_model`: `True` if the model takes raw data (e.g. pixels, waveforms) as input, not hand-crafted features.
- `has_separate_manual_feature_stage`: `True` if there is a separate, hand-written feature extraction stage **outside** the neural network.
- `has_multiple_independent_models`: `True` if the pipeline consists of several independently trained models chained together (not one jointly trained model).

Rules (for this homework):
- A pipeline is **end-to-end** if:
  - `has_raw_input_to_model` is `True`, AND
  - `has_separate_manual_feature_stage` is `False`, AND
  - `has_multiple_independent_models` is `False`.
- Otherwise it is **not** end-to-end.

Return `True` if the pipeline is end-to-end, otherwise `False`.

In [10]:
# TODO: implement is_end_to_end
def is_end_to_end(has_raw_input_to_model: bool,
                  has_separate_manual_feature_stage: bool,
                  has_multiple_independent_models: bool) -> bool:
    """Return True if the pipeline is end-to-end deep learning (according to the rules above)."""
    # YOUR CODE HERE
    raise NotImplementedError


In [11]:
# TEST CELL: Task 3 (30 points)
max_points = 30
points = 0

try:
    # 1) Pure end-to-end: raw input, no manual features, single model
    if is_end_to_end(True, False, False) is True:
        points += 10
    else:
        print('Case 1 failed: should be end-to-end.')

    # 2) Manual features present -> not end-to-end
    if is_end_to_end(True, True, False) is False:
        points += 5
    else:
        print('Case 2 failed: manual feature stage -> not end-to-end.')

    # 3) Multiple independently trained models -> not end-to-end
    if is_end_to_end(True, False, True) is False:
        points += 5
    else:
        print('Case 3 failed: multiple independent models -> not end-to-end.')

    # 4) No raw input to model -> not end-to-end
    if is_end_to_end(False, False, False) is False:
        points += 5
    else:
        print('Case 4 failed: no raw input to model -> not end-to-end.')

    # 5) All bad flags -> not end-to-end
    if is_end_to_end(False, True, True) is False:
        points += 5
    else:
        print('Case 5 failed: all flags true -> not end-to-end.')

except Exception as e:
    print('Error when calling is_end_to_end:', e)

_set_score('3', points, max_points)


Error when calling is_end_to_end: 
Task 3: 0 / 30 points
TOTAL POINTS: 0 / 100



In [12]:

max_total = max_points
total_points=points
print(f"Final score: {total_points}/{max_total}")

Final score: 0/30


In [13]:
import json
import numpy as np
from datetime import datetime, timezone

# ---------- тестовые помощники ----------
def add_points(ok, pts, msg_ok, msg_fail):
    global total
    if ok:
        total += pts
        print(f"[+{pts:>2}] {msg_ok}")
    else:
        print(f"[ 0] {msg_fail}")

def _sec(td):
    return td.total_seconds()

def penalty_fraction(start_dt, due_dt, now_dt) -> float:
    if not (start_dt and due_dt and now_dt):
        return 0.0
    window = _sec(due_dt - start_dt)
    if window <= 0:
        return 1.0 if now_dt > due_dt else 0.0
    late = max(0.0, _sec(now_dt - due_dt))
    return min(1.0, late / window)
# ---------- автогрейдинг ----------
total = 0
max_points = max_total
raw_score = total_points

import json

# применяем штраф
try:
    pf = penalty_fraction(start_dt, due_dt, submission_dt)
except NameError:
    from datetime import timezone
    pf = 0.0
# ✅ Итоговый результат

max_points = 100

raw_score = round(100.0 * total / max_points)
final_score = max(0.0, raw_score * (1.0 - min(1.0, pf)))

print(f"Сырой балл: {raw_score}/{max_points}")
print(f"Штраф (доля): {pf:.4f}")
print(f"Итоговый балл после штрафа: {final_score:.2f}/{max_points}")

# Последняя строка — JSON, который читает harness
final = {
    "name": full_name,
    "group": student_group,
    "assignment": assignment_id,
    "score": float(final_score)
}


Сырой балл: 0/100
Штраф (доля): 0.0000
Итоговый балл после штрафа: 0.00/100


In [14]:
print(json.dumps(final, ensure_ascii=False))

{"name": "Doe John", "group": "11-111", "assignment": "EM02_Keras", "score": 0.0}


## Final note

You can re-run the test cells after changing your solutions.
The final **TOTAL POINTS** is printed after each test cell.

Good luck!